# Microarray data analysis

Microarray data in genetics refers to a type of high-throughput technology used to measure the expression levels of thousands of genes simultaneously. Microarrays are composed of small glass slides or silicon chips on which thousands of microscopic spots, known as probes, are deposited in an orderly grid pattern. These probes are short DNA sequences or fragments that are complementary to specific genes or regions of the genome. Each spot represents a particular gene or DNA sequence. The results are typically strored in a .CEL file. To process a .CEL file we go thourgh the following steps

- Get .CEL from https://www.ncbi.nlm.nih.gov/
- Every .CEL file corresponds to an experiment which is part of SERIES and SERIES is a based on a platform.
- We use GPL570 platform to restrict our-selves to **Affymetrix Human Genome U133 Plus 2.0 Array**

### Collecting HTML files of sample description

- Download the list of all accession of GPL570 at https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GPL570&targ=self&view=brief&form=text

In [ ]:
# Read all sample_ids into a list:

with open('GPL570.txt','r') as f:
    lines = f.readlines()
    sample_id = []
    for line in lines:
        if line[:22] == '!Platform_sample_id = ':
            sample_id.append('GSM'+rx.findall(r'\d+',line)[0])

In [ ]:
import requests as re
from bs4 import BeautifulSoup
import pandas as pd
import regex as rx

# Download all html files corresponding to the sample ids.

for idx in sample_id:
    url = 'https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc='+idx
    page = re.get(url)
    with open('./html_files/'+idx+'.html', 'wb') as file:
        file.write(page.content)

### Extract the Age and Sex information from the HTML files 

In [ ]:
def get_age_sex(nn):
    '''
    This function simply match patterns in the text where gender is explicitly 
    defined and age is mentioned as "Age". However, we have encountered there  
    are  many other types of Sex/Age information like : F/26 and others. We 
    intentionally did not make it complicated, since processing too many files 
    in laptop is beyond its capacity. 
    '''
    with open('./html_files/'+sample_id[nn]+'.html','r') as f:
        page = f.read()
    soup = BeautifulSoup(page,'html.parser')
    text = soup.get_text(separator=' ')
    loc = text.find(' Age')
    if loc > 0:
        age = rx.findall(r'\d+', text[loc:loc+20])
    else:
        age = np.nan
        
    sex_list = rx.findall(r'(Male|Female|male|female)\s+', text)
    if len(sex_list) > 0:
        sex = sex_list[0]
    else:
        sex = np.nan
    return age, sex


## Parallelised extraction of age/asex information

pool = Pool() ### number of processesor I want to use
out = zip(*pool.map(get_age_sex, range(169340)))
age_sex_list = list(out)


# Arrange ages with Minimum(age) and Maximum(Age) information 

age_list = age_sex_list[0]
sex_list = age_sex_list[1]

age_list_1 = []
age_list_2 = []

for ages in age_list:
    if type(ages) is not list:
        age1 = ages
        age2 = ages
    elif len(ages) > 1:
        age1 = ages[0]
        age2 = ages[1]
    elif len(ages) == 1:
        age1 = ages[0]
        age2 = ages[0]
    else:
        age1 = np.nan
        age2 = np.nan
    age_list_1.append(age1)
    age_list_2.append(age2)


df = pd.DataFrame({'sample_id' : sample_id, 'age_1': age_list_1, 'age_2':age_list_2 , 'sex': sex_list})

## Write the result in a file
df_filtered = df.dropna()
df_filtered.to_csv('age_sex_data.csv')

### Collecting the .CEL files

We download the .CEL files corresponding to the sample_ids whose age/sex infromation is available

In [2]:
# Write a txt file which contains all the downlaod links

for idx in df_filtered.sample_id:
    with open('./html_files/'+idx+'.html','r') as f:
        page = f.read()
    soup = BeautifulSoup(page, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href[:3] == 'ftp':
            with open('./cel_files/cel_links.txt','a') as ff:
                ff.write(href+'\n')

Download the cell files using bash script

```bash
#!/bin/bash

count=0  # Initialize a variable to keep track of the number of files downloaded

# Read the file line by line
while IFS= read -r link
do
    # Use wget to download each file
    wget "$link"
    ((count++))  # Increment the count after each successful download
done < cel_links.txt

# Output the number of files downloaded as a comment
echo "Number of files downloaded: $count"

```

### fRMA Normaization of .CEL files

Frozen RMA (fRMA) is a microarray preprocessing algorithm that allows one to analyze
microarrays individually or in small batches and then combine the data for analysis. This
is accomplished by utilizing information from the large publicly available microarray
databases. Specifically, estimates of probe-specific effects and variances are precomputed
and frozen. Then, with new data sets, these frozen parameters are used in concert with
information from the new array(s) to preprocess the data.
Follow documentation: https://www.bioconductor.org/packages/release/bioc/html/frma.html

The following **R-script** does the fRMA transformation and saves output of z-scores in .csv format. 

```R
library(frma)
library(affy)


input_path = paste0('.../cel_files/',i,'.CEL.gz')
Data <- ReadAffy(filenames = input_path)

# for custom CDF file use:
# Data <- ReadAffy(filenames = input_path, cdfname = '...path_to/GPL17996_HGU133Plus2_Hs_ENTREZG.cdf')


Object <- frma(Data)
   bc<- barcode(Object, output = 'z-score')
   output_path = paste0('..../expr_files/',i,'.csv')
   write.csv(bc, file = output_path)}
```

For processing all the .cel files in a parallelized for loop look at **get_z_score_parallel.r**